In [ ]:
import os
import re
import random
import numpy as np
import pandas as pd
import scipy.io
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import manifold_dynamics.io_matlab_s3 as ims
import manifold_dynamics.neural_utils as nut

In [ ]:
datadir = '../datasets/NNN/'
fnames = ims.fnames(datadir)
raster_data = pd.read_pickle(os.path.join(datadir, ('unit_data_full.pkl')))
fr_data = pd.read_pickle(os.path.join(datadir, 'fr_data.pkl'))

In [ ]:
raster_data['roi'].unique()

In [ ]:
metric = 'correlation'
roi = 'CLC3_13_O' # 'MF1_9_F', 'PITP4_10_O', 'CLC3_13_O'

roi_data = raster_data[(raster_data['roi']==roi)]
stacked = np.stack(roi_data['img_psth'])

fr_roi = fr_data[(fr_data['roi']==roi)]
out = nut.derag_fr(fr_roi, period='early')

In [ ]:
# calculate the mean firing rate across trials for each unit, image
mean_firing = out.mean(axis=2)

# sort by mean_firing for each unit
sort_indices = np.argsort(mean_firing, axis=1)[:, ::-1]

# broadcast to shape (units, 1, images)
sort_indices_expanded = sort_indices[:, np.newaxis, :]

stacked_sorted = np.take_along_axis(stacked, sort_indices_expanded, axis=2)

In [ ]:
topn = 100

# plot time x time
substacked = stacked_sorted[:, :, :topn]
# average over units
averaged = np.mean(substacked, axis=0)

fig,ax = plt.subplots(1,1)

rdm = squareform(pdist(averaged, metric=metric))
sns.heatmap(rdm, cmap=sns.color_palette('Greys_r', as_cmap=True), vmax=1.5)
ax.set_title(f'{roi}: top {topn} nsd')

ax.axvline(x=50, color='red', linestyle='--', linewidth=1)
ax.axhline(y=50, color='red', linestyle='--', linewidth=1)
plt.show()